In [1]:
# Import libraries to interact with Google Drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:

# Authenticate with your Google account to get access to the data
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:

# Download data
download = drive.CreateFile({'id': '1rw5l3jCo2vlNc8NLrMk3KPZr6HsjNRCY'})
download.GetContentFile('sonar.csv')

In [4]:
ls

sample_data/  sonar.csv


In [5]:
import pandas as pd


# Ruta al archivo CSV en tu Google Drive
ruta_archivo = '/content/sonar.csv'

# Lee el archivo CSV
df = pd.read_csv(ruta_archivo)

# Ahora puedes trabajar con tu DataFrame
print(df.head())


   0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109  \
0  0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
1  0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
2  0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
3  0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   
4  0.0286  0.0453  0.0277  0.0174  0.0384  0.0990  0.1201  0.1833  0.2105   

   0.2111  ...  0.0027  0.0065  0.0159  0.0072  0.0167  0.0180  0.0084  \
0  0.2872  ...  0.0084  0.0089  0.0048  0.0094  0.0191  0.0140  0.0049   
1  0.6194  ...  0.0232  0.0166  0.0095  0.0180  0.0244  0.0316  0.0164   
2  0.1264  ...  0.0121  0.0036  0.0150  0.0085  0.0073  0.0050  0.0044   
3  0.4459  ...  0.0031  0.0054  0.0105  0.0110  0.0015  0.0072  0.0048   
4  0.3039  ...  0.0045  0.0014  0.0038  0.0013  0.0089  0.0057  0.0027   

   0.0090  0.0032  R  
0  0.0052  0.0044  R  
1  0.0095  0.0078  R  
2  0.0040  0.0117  R  


In [31]:
# cargo los datos
df = pd.read_csv('/content/sonar.csv', header=None)

# pongo nuevas etiquetas a las columnas
titulos = [f'Columna_{i}' for i in range(1, 62)]  # Del 1 al 61

#
df.columns = titulos

# Convertir 'R' a 0 y 'M' a 1 en la columna de etiquetas
df['Etiquetas'] = df['Columna_61'].apply(lambda x: 0 if x == 'R' else 1)


df.head()

,Columna_1,Columna_2,Columna_3,Columna_4,Columna_5,Columna_6,Columna_7,Columna_8,Columna_9,Columna_10,...,Columna_53,Columna_54,Columna_55,Columna_56,Columna_57,Columna_58,Columna_59,Columna_60,Columna_61,Etiquetas
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R,0
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R,0
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R,0
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R,0
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R,0


In [35]:
# Eliminar la columna original de etiquetas
df.drop('Columna_61', axis=1, inplace=True)

In [36]:
#divido los datos en etiquetas X Y para poder entrenar la red
X = df.drop('Etiquetas', axis=1).values
y = df['Etiquetas'].values

In [37]:
# Normalizar los datos para entrenar correctamente la red
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_val, y_val), verbose=2)

# Evaluación del modelo
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {accuracy:.2f}')

Epoch 1/50
11/11 - 1s - loss: 0.6680 - accuracy: 0.5663 - val_loss: 0.6033 - val_accuracy: 0.6190 - 1s/epoch - 107ms/step
Epoch 2/50
11/11 - 0s - loss: 0.5630 - accuracy: 0.7530 - val_loss: 0.5256 - val_accuracy: 0.7381 - 51ms/epoch - 5ms/step
Epoch 3/50
11/11 - 0s - loss: 0.4899 - accuracy: 0.8313 - val_loss: 0.4801 - val_accuracy: 0.7381 - 66ms/epoch - 6ms/step
Epoch 4/50
11/11 - 0s - loss: 0.4301 - accuracy: 0.8554 - val_loss: 0.4459 - val_accuracy: 0.7857 - 53ms/epoch - 5ms/step
Epoch 5/50
11/11 - 0s - loss: 0.3780 - accuracy: 0.8855 - val_loss: 0.4194 - val_accuracy: 0.7619 - 66ms/epoch - 6ms/step
Epoch 6/50
11/11 - 0s - loss: 0.3304 - accuracy: 0.8976 - val_loss: 0.4058 - val_accuracy: 0.7857 - 52ms/epoch - 5ms/step
Epoch 7/50
11/11 - 0s - loss: 0.2914 - accuracy: 0.9337 - val_loss: 0.3947 - val_accuracy: 0.8095 - 71ms/epoch - 6ms/step
Epoch 8/50
11/11 - 0s - loss: 0.2584 - accuracy: 0.9458 - val_loss: 0.3837 - val_accuracy: 0.7857 - 56ms/epoch - 5ms/step
Epoch 9/50
11/11 - 0s - 